In [1]:
%load_ext autoreload
%autoreload 2
from sound_field import SoundField,SoundFieldDataset,divide_to_subbands,divide_to_time_windows
from signal_info import signal_info
from optimizer import optimizer
from optimizer_v2 import optimizer_v2
from DoA_est import DoA_via_bands
import utils
import numpy as np
import math
import time
import torch
import gc
import os
from collections import defaultdict,Counter
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader


LEBEDEV = 'lebedev'
POINTS_162 = '162_points'
SQP_OPT_METHOD = "SQP"
GD_DEEP_OPT_METHOD = "GD_Deep"
GD_OPT_METHOD = "GD_lagrange_multi" #GD_Deep GD_lagrange_multi
SLS_OPT_METHOD = "SLS"
if torch.cuda.is_available():
    DEVICE = torch.device("cuda:0")
    print("Device Name:", torch.cuda.get_device_name(DEVICE))
else:
    DEVICE = torch.device("cpu")
    print("Device Name: CPU")
torch.set_grad_enabled(False)
torch.cuda.empty_cache()  # Free up unused memory on GPU
gc.collect()  # Clean up unused memory on CPU


Device Name: NVIDIA GeForce RTX 4080 SUPER


56

In [2]:
data_set = torch.load(r"data\WSJ0\Dataset_03_01_25.pt",weights_only=False)

In [3]:
def custom_collate_fn(batch):
    return batch  # Returns a list of custom objects

In [4]:
train_set = SoundFieldDataset(data_set['train'])
val_set = SoundFieldDataset(data_set['validation'])
test_set = SoundFieldDataset(data_set['test'])

In [5]:
batch_size = 1
num_epochs = 100

down_sample = 1
num_bins = 45

window_length = 1024
max_num_windows = int(1e10)

mask = None

train_data_loader = DataLoader(train_set, batch_size=batch_size,collate_fn=custom_collate_fn, shuffle=True)
val_data_loader = DataLoader(val_set, batch_size=len(val_set),collate_fn=custom_collate_fn, shuffle=True)
test_data_loader = DataLoader(test_set, batch_size=len(test_set),collate_fn=custom_collate_fn, shuffle=True)

Y_p = utils.create_sh_matrix(train_set.input_order, zen=train_set[0].P_th, azi=train_set[0].P_ph).to(DEVICE)
opt_model = optimizer_v2(Y_p=Y_p,alpha=0.01,num_iters=5,device=DEVICE)


In [6]:
Y_p.shape

torch.Size([4, 162])

In [7]:
for epoch in range(num_epochs):
    for batch in train_data_loader:
        curr_sound_field = batch[0]
        curr_sound_field.a_nm_subbands = divide_to_subbands(anm_t=curr_sound_field.anm_t, num_bins=num_bins, downsample=down_sample, sr=curr_sound_field.sr)
        curr_sound_field.windowed_anm_t = divide_to_time_windows(anm_t_subbands=curr_sound_field.a_nm_subbands,window_length=window_length, max_num_windows=max_num_windows)
        
        curr_sound_field.num_bins = num_bins
        curr_sound_field.window_length = window_length
        curr_sound_field.num_windows = curr_sound_field.windowed_anm_t.shape[0]
        est_upscaled_subbands = opt_model.forward(curr_sound_field,mask)
        
        gt_upscale_subbands = divide_to_subbands(anm_t=curr_sound_field.gt_anm_t_upscaled, num_bins=num_bins, downsample=down_sample,sr=curr_sound_field.sr)
        gt_upscale_subbands_windowed = divide_to_time_windows(anm_t_subbands=gt_upscale_subbands,window_length=window_length, max_num_windows=max_num_windows)

    break

100%|██████████| 5/5 [00:00<00:00, 23.15it/s]


cuda:0 cuda:0


OutOfMemoryError: CUDA out of memory. Tried to allocate 66.83 GiB. GPU 0 has a total capacity of 15.99 GiB of which 11.81 GiB is free. Of the allocated memory 270.53 MiB is allocated by PyTorch, and 2.53 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)